In [1]:
import pandas as pd
import numpy as np
products_static = pd.read_csv("products_static_1607946056.csv")

# INPUTS: 
Input the values you would like to filter by:

In [2]:
#Filter 1: Categories
(cat_1_on, cat_2_on, cat_3_on) = False, True, False
cat_lst_1 = []
cat_lst_2 = [12999, 1301, 21762, 1297, 1295, 17095, 893, 
              17089, 12738, 2037,2035, 9197, 20768, 2033, 
              1329, 14492, 14503, 21849, 16502, 16500]
cat_lst_3 = []

#Filter 2: Times (Update-status, how long been sold)
time_on = True
unix_time_now = 1607946056 #Change based on Nick's input
min_years = 1
updated_previously_weeks = 2


#Filter 3: Brand GMV
min_gmv_on = False
min_gmv = 1000

#Filter 4: Brand Avg Price 
min_unit_price_on = True
min_unit_price = 300

#Filter 5: GMV Concentration
min_gmv_conc_on = False
top_n = 3 #concetration num of products
min_concentration = 0.2

#Filter 6: Avg Product Rating
min_avg_star_on = False
min_avg_star = 4.5

#Filter 7: Bad Customer Ratings 
max_bad_rating_on = False 
max_bad_rating = 0.5

#Filter 8: Brand TOTAL GMV 
min_total_brand_gmv_on = True
min_total_brand_gmv = 2000000 / 12

# Import Data

In [3]:
shops_gmv = pd.read_csv("shops_gmv_data.csv")
products_static = pd.read_csv("products_static_1607946056.csv")
models_static = pd.read_csv("models_static_data.csv")

Change data types in dataframes to conserve memory:

In [4]:
def convert_to_cats(df, to_convert):
    '''
    Converts cols in to_convert_list to categories. 
    
    Inputs:
        df: a DataFrame
        to_convert: a list of column names (as strings)
                    we'd like to convert
    Returns:
        Nothing; modifies dataframe in place
    '''
    for col in df.columns:
        if col in to_convert:
            df[col] = df[col].astype("category")

In [5]:
to_convert_models = ['category_one', 'category_one_en',
                     'category_one_th', 'category_two',
                     'category_two_en', 'category_two_th',
                     'category_three', 'category_three_en',
                     'category_three_th']
convert_to_cats(models_static, to_convert_models)

to_convert_products = ['category_one', 'category_one_en',
                      'category_one_th', 'category_two', 
                      'category_two_en', 'category_two_th',
                      'category_three', 'category_three_en', 
                      'category_three_th', 'reviews_count_context', 
                      'reviews_count_image', 'shopee_verified', 
                      'show_discount']
convert_to_cats(products_static, to_convert_products)

to_convert_shops = ["shopid", "category_one", "category_two",
                    "category_three"]
convert_to_cats(shops_gmv, to_convert_shops)

In [6]:
def downcast_numbers(df):
    '''
    Downcasts floats and ints.
    
    Inputs:
        df: a DataFrame objecet. 
    Returns:
        Nothing; modifies list in place
    '''
    for col in df.columns:
        if df[col].dtype == "float":
            df[col] = pd.to_numeric(df[col], downcast="float")
        if df[col].dtype == "int":
            df[col] = pd.to_numeric(df[col], downcast="unsigned")

In [7]:
downcast_numbers(models_static)
downcast_numbers(products_static)
downcast_numbers(shops_gmv)

#For Step 9:
products_static["product_gmv"] = products_static["price"] * products_static["sold"] 
products_static_untouched = products_static.copy(deep=True)


# Filter 1: Category Filter

Filter our DataFrames to only keep rows with Categories we care for. 


In [8]:
def filter_by_category(df, category_n, ok_cat_lst):
    '''
    Keeps rows in the df that have cats in ok_cat_lst. 
    
    Inputs:
        df: a DataFrame
        category_n: (str) eg. "category_one"
        ok_cat_lst: (lst) of acceptable cat for that 
          cat as numbers eg [50, 26]
    Returns:
        Nothing; modifies list in place
    '''
    
    mask = df[category_n].isin(ok_cat_lst)
    df = df[mask]

In [9]:
def filter_by_category(df, category_n, ok_cat_lst):
    '''
    Keeps rows in the df that have cats in ok_cat_lst. 
    
    Inputs:
        df: a DataFrame
        category_n: (str) eg. "category_one"
        ok_cat_lst: (lst) of acceptable cat for that 
          cat as numbers eg [50, 26]
    Returns:
        The modified df 
    '''
    
    mask = df[category_n].isin(ok_cat_lst)
    return df[mask]

In [10]:
if cat_1_on:
    models_static = filter_by_category(models_static, "category_one", cat_lst_1)
    products_static = filter_by_category(products_static, "category_one", cat_lst_1) 

In [11]:
if cat_2_on:
    models_static = filter_by_category(models_static, "category_two", cat_lst_2)
    products_static = filter_by_category(products_static, "category_two", cat_lst_2) 

In [12]:
if cat_3_on:
    models_static = filter_by_category(models_static, "category_three", cat_lst_3)
    products_static = filter_by_category(products_static, "category_three", cat_lst_3) 

# Filter 2: Keep only Established Products and Products that Sellers are Updating

Filter out products that have been sold for less than a specified time. 

In [13]:
if time_on: 
    from datetime import datetime
    import math 

    def filter_out_new_products(df, min_years, unix_time_now):
        '''
        Filters out rows from our products_static df that 
        have been sold less than a specified time
        '''

        min_s = min_years * 365 * 24 * 60 * 60 

        mask = (unix_time_now - products_static["ctime"] > min_s)

        return df[mask]

    products_static = filter_out_new_products(products_static, min_years,
                                              unix_time_now)

    def filter_out_unupdated_products(df, updated_previously_weeks):
        '''
        Filters out rows from our products_static df that 
        have been sold less than a specified time
        '''

        unix_time_now = math.ceil(datetime.now().timestamp())

        max_s = updated_previously_weeks * 7 * 24 * 60 * 60

        mask = (unix_time_now - products_static["modified_at"] < max_s)

        return df[mask]

    products_static = filter_out_unupdated_products(products_static, 
                                                    updated_previously_weeks)

# Brand Definition

Extract unique brand names from the remaining products

In [14]:
mask = products_static["brand"] != "No Brand(ไม่มียี่ห้อ)"
products_static = products_static[mask]

brands_list = products_static["brand"].unique()
brands_list = np.delete(brands_list, np.where(brands_list == ('No Brand(ไม่มียี่ห้อ)')))

In [15]:
#Clean up brands_df
brands_df = pd.DataFrame(brands_list, columns = ["Brand Name"])

brands_df.dropna(inplace=True)

s1 = brands_df["Brand Name"] != "None"
brands_df = brands_df[s1]

s2 = brands_df["Brand Name"] != "0"
brands_df = brands_df[s2] 


In [16]:
#Set up columns for our brands_df
brands_df["Brand_GMV"] = 0
brands_df["Average_Unit_Price"] = 0
brands_df["GMV_Concentration"] = 0
brands_df["Weighted_Star_Rating"] = 0
brands_df["Bad_Rating_Percent"] = 0
brands_df.set_index("Brand Name", inplace = True)

In [17]:
#Make new column in products_static for GMV of a product (price * sold)
products_static["weighted_star"] = products_static["rating_star"] * products_static["sold"]

In [18]:
for brand in brands_df.index:
    
    #make a sub-df containing only rows with the correct brand
    my_brand_df = products_static[products_static["brand"] == brand]
    my_brand_df = my_brand_df.sort_values(by=["product_gmv"], ascending=False) 
    
    #gmv calculation
    brand_gmv = my_brand_df["product_gmv"].sum()
    brands_df.loc[brand, "Brand_GMV"] = brand_gmv
    
    #volume calculation
    brand_volume = my_brand_df["sold"].sum()
    brands_df.loc[brand, "Average_Unit_Price"] = brand_gmv / brand_volume
    
    #GMV Concentration Calculation
    if len(my_brand_df) >= top_n: 
        top_n_df = my_brand_df.head(top_n)
        top_n_gmv = top_n_df["product_gmv"].sum()
        brands_df.loc[brand, "GMV_Concentration"] = top_n_gmv / brand_gmv
    else:
        brands_df.loc[brand, "GMV_Concentration"] = 1.0 #brand sells less than like 5 products --> highly conc.
        
    #Weighted Star Rating
    total_star = my_brand_df["weighted_star"].sum()
    brands_df.loc[brand, "Weighted_Star_Rating"] = total_star / brand_volume
    
    #Bad Rating Count
    bad_rating_count = my_brand_df["rating_count_one"].sum() + my_brand_df["rating_count_two"].sum()
    total_rating_count = my_brand_df["rating_count_total"].sum()
    
    if total_rating_count != 0:
        val = bad_rating_count / total_rating_count 
    else: 
        val = -1
    
    brands_df.loc[brand, "Bad_Rating_Percent"] = val

brands_df["Bad_Rating_Percent"] = brands_df["Bad_Rating_Percent"].round(decimals=3)

# Filter 3: Brand Total GMV

In [19]:
if min_gmv_on:
    brands_df = brands_df[brands_df["Brand_GMV"] >= min_gmv]

# Filter 4: Brand Average Price

In [20]:
if min_unit_price_on:
    brands_df = brands_df[brands_df["Average_Unit_Price"] >= min_unit_price]

# Filter 5: GMV Concentration

In [21]:
if min_gmv_conc_on:
    brands_df = brands_df[brands_df["GMV_Concentration"] >= min_concentration]

# Filter 6: Average Product Rating

In [22]:
if min_avg_star_on:
    brands_df = brands_df[brands_df["Weighted_Star_Rating"] >= min_avg_star]

# Filter 7: Bad Customer Ratings

In [23]:
if max_bad_rating_on:
    brands_df = brands_df[brands_df["Bad_Rating_Percent"] <= max_bad_rating]

# (GMV Growth?)

In [24]:
# number of brands in our brands_df
print(len(brands_df))

2573


# Filter 8: Brand GMV (Across All Categories) 

Reimport the products data but only filter for the brands in our df. Note: We do this step at the end b/c we'll have to reimport all of products data, so makes sense to have the smallest brands list possible. 

### Calculate Category of the Brands

Do this by looping over again, since going through each cat 2 for each brand is computationally expensive, want to get the brands list down as much as possible.
Note: 
- If set min_total_brand_gmv = 2,000,000 / 12, go from 415 --> 415
- If set min_total_brand_gmv = 2,000,000 / 10, go from 415 --> 364
- If set min_total_brand_gmv = 2,000,000 / 8, go from 415 --> 314
- If set min_total_brand_gmv = 2,000,000 / 6, go from 415 --> 258
- If set min_total_brand_gmv = 2,000,000 / 4, go from 415 --> 193

This is just a sample but goes to show that doing our brand's cat_2 calculation can be expensive, so that's why I wanted to put it after we had chipped away at list. However, if we set min_total_brand_gmv = 2,000,000 / 12, there's no effect so might as well just include our brand cat 2 calculation in Filter 8's for loop. 


In [25]:
#Assume this is always kept on for simplicity
if min_total_brand_gmv_on:
        
    for brand in brands_df.index:

        #make a sub-df containing only rows with the correct brand
        mask_brand = products_static_untouched["brand"] == brand
        my_brand_df = products_static_untouched[mask_brand]

        #TOTAL Brand gmv calculation
        total_brand_gmv = my_brand_df["product_gmv"].sum()
        brands_df.loc[brand, "Brand_GMV_All_Cats"] = total_brand_gmv
        
        #Total brand view count 
        total_brand_views = my_brand_df["view_count"].sum()
        conv_rate = total_brand_gmv / total_brand_views
        brands_df.loc[brand, "Conversation_Rate_All_Cats"] = conv_rate
        
        #--------------------------------------------------------------
        #Take this out and put later if the min_total_brand_gmv 
        #becomes higher, otherwise inclue this in the for loop
        winning_cat_2 = None
        winning_cat_2_gmv = 0.0

        for cat_2 in my_brand_df["category_two_en"].unique():
            mask_cat_2 = my_brand_df["category_two_en"] == cat_2
            cat_2_df = my_brand_df[mask_cat_2]

            cat_2_gmv = cat_2_df["product_gmv"].sum()
            if cat_2_gmv > winning_cat_2_gmv:
                winning_cat_2 = cat_2
                winning_cat_2_gmv = cat_2_gmv
                    
        percent = (winning_cat_2_gmv / total_brand_gmv).round(decimals=3)
        brands_df.loc[brand, "Brand_Cat_2_Contr"] = percent 
        
        brands_df.loc[brand, "Brand_Category_2"] = winning_cat_2
        
        
        print("for brand: ", brand, "Winning cat 2 is: ", winning_cat_2)
        print("with a cat_2_gmv value of: ", winning_cat_2_gmv)

        
        #--------------------------------------------------------------
            
        
        
    #Apply the total brand gmv filter
    brands_df = brands_df[brands_df["Brand_GMV_All_Cats"] >= min_total_brand_gmv]
    
brands_df = brands_df[brands_df["Brand_GMV_All_Cats"] >= min_total_brand_gmv]

for brand:  Genie Bra(จีนี่ บรา) Winning cat 2 is:  Underwear
with a cat_2_gmv value of:  200022
for brand:  PORTABLE TOILET(พอร์ทเทเบิล ทอยเลท) Winning cat 2 is:  Camping and hiking
with a cat_2_gmv value of:  130130
for brand:  TVDirect(ทีวีไดเร็กท์) Winning cat 2 is:  Health equipment
with a cat_2_gmv value of:  2580
for brand:  SKG(เอสเคจี) Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  1182053
for brand:  Bandai(บันได)​ Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1892788
for brand:  BANDAI(บันได) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  366985
for brand:  GENA(จีน่า) Winning cat 2 is:  Underwear
with a cat_2_gmv value of:  980
for brand:  MaxDe Winning cat 2 is:  Furniture
with a cat_2_gmv value of:  64596
for brand:  Bandai(บันได) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  260032
for brand:  Colorland(คัลเลอร์แลนด์) Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  20220
for brand:

for brand:  Smarthome (สมาร์ทโฮม) Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  712406
for brand:  Luv Care (เลิฟ แคร์) Winning cat 2 is:  Dog
with a cat_2_gmv value of:  199516
for brand:  SmartHeart Gold(สมาร์ทฮาร์ทโกลด์) Winning cat 2 is:  Dog
with a cat_2_gmv value of:  523406
for brand:  Dr.luvcare(ดร.ลูวแคร์) Winning cat 2 is:  Dog
with a cat_2_gmv value of:  65785
for brand:  Maxde Winning cat 2 is:  Kitchen and dining room
with a cat_2_gmv value of:  45052
for brand:  Smart heart(สมาร์ท ฮาร์ท) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  14770
for brand:  Lotus Midas(โลตัส ไมดาส) Winning cat 2 is:  Bedroom
with a cat_2_gmv value of:  77310
for brand:  King Pet(คิงเ พ็ท) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  7050
for brand:  Smartheart Gold Winning cat 2 is:  Cat
with a cat_2_gmv value of:  3570
for brand:  Me-O Gold(มีโอ โกลด์) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  25946
for brand:  Zojirushi(โซจิรุชิ) Winning cat 2 is: 

for brand:  Osprey Games Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1400
for brand:  Iello(เอลโล) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  3400
for brand:  Breaking Games Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1400
for brand:  INDEX LIVING Winning cat 2 is:  Furniture
with a cat_2_gmv value of:  27590
for brand:  โนแบรนด์ (No Brands) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  950
for brand:  Floodgate Games Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  800
for brand:  Sgarlet by Wacoal(สกาเล็ต บาย วาโก้) Winning cat 2 is:  Underwear
with a cat_2_gmv value of:  386175
for brand:  WINNER FURNITURE(วินเนอร์ เฟอร์นิเจอร์) Winning cat 2 is:  Furniture
with a cat_2_gmv value of:  13140
for brand:  The cool(เดอะ คูล) Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  1950
for brand:  Autobot(ออโต้บอท) Winning cat 2 is:  Vacuum cleaners and floor care equipment
with a cat_2_g

/Users/dassamongkol/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in long_scalars
  app.launch_new_instance()


for brand:  USAOPOLY Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1200
for brand:  Repos Production(รีโพส โปรดักชั่น) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1800
for brand:  Repos Production(รีพอส โปรดักชั่น) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1200
for brand:  HeidelBÄR Games Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1600
for brand:  DUNLOP(ดันล๊อป) Winning cat 2 is:  Racket sports
with a cat_2_gmv value of:  35419
for brand:  Atttitude Mom(แอ็ดติจูด) Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  48082
for brand:  Attitude mom(แอดติจูด มัม) Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  231540
for brand:  Attitude mom Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  700
for brand:  Attitude Mom Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  1950
for brand:  Attitude Mom(แอทติจูด มัม) Winning cat 2 is:  Equipment for children
with a cat_2_gmv valu

...

for brand:  TFP(ทีเอฟพี) Winning cat 2 is:  Car accessories
with a cat_2_gmv value of:  8377
for brand:  Sun​ Story​(ซันสตอรี่) Winning cat 2 is:  Furniture - Children mattress
with a cat_2_gmv value of:  1489
for brand:  Junction Produce(จัคชั่น โปรดักซ์) Winning cat 2 is:  Car exterior accessories
with a cat_2_gmv value of:  3160
for brand:  อ้วนออเงิน Winning cat 2 is:  Car exterior accessories
with a cat_2_gmv value of:  5118
for brand:  TGR Winning cat 2 is:  Car accessories
with a cat_2_gmv value of:  3678
for brand:  Dekk Dekk(เด๊ก เด๊ก) Winning cat 2 is:  Equipment for children
with a cat_2_gmv value of:  390
for brand:  Harada(ฮาราดา) Winning cat 2 is:  Car exterior accessories
with a cat_2_gmv value of:  428
for brand:  Zarbees(ซาบี) Winning cat 2 is:  Equipment for health and safety
with a cat_2_gmv value of:  19462
for brand:  Natures Aid(เนเจอรส์เอด) Winning cat 2 is:  Equipment for health and safety
with a cat_2_gmv value of:  790
for brand:  Lisa(ลิซ่า) Winning cat 2 is:

for brand:  WD-40 Winning cat 2 is:  Power tools and tools
with a cat_2_gmv value of:  12332
for brand:  kar Winning cat 2 is:  Car care products
with a cat_2_gmv value of:  1510
for brand:  Nanonix Winning cat 2 is:  Car care products
with a cat_2_gmv value of:  1990
for brand:  Caltex(คาลเท็กซ์) Winning cat 2 is:  Oil and liquid
with a cat_2_gmv value of:  587099
for brand:  itp Winning cat 2 is:  Car care products
with a cat_2_gmv value of:  1975
for brand:  SWEETPEA(สวีทพี) Winning cat 2 is:  Baby goods
with a cat_2_gmv value of:  7450
for brand:  Denpa Winning cat 2 is:  Kitchen and dining room
with a cat_2_gmv value of:  139887
for brand:  Taito(ไทโตะ) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  5886
for brand:  Sr(เอสอาร์) Winning cat 2 is:  Furniture
with a cat_2_gmv value of:  40680
for brand:  Origen Winning cat 2 is:  Cat
with a cat_2_gmv value of:  5970
for brand:  Meo(มีโอ) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  4794
for brand:  Dodopet Win

for brand:  LOL Surprise(แอลโอแอล เซอร์ไพรส์) Winning cat 2 is:  Doll
with a cat_2_gmv value of:  29964
for brand:  Marvel Select(มาเวล ซีเลค) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  6950
for brand:  Marvel Legends(มาเวลลีเจ้นท์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  2250
for brand:  S.H.Figuarts(เอสเอช ฟิกเกอร์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  11040
for brand:  ThinkFun(ธิงค์ฟัน) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  3150
for brand:  Gigamic Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  600
for brand:  Totty Books(ท๊อตตี้บุ๊คส์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  5515
for brand:  Iron Studios(ไอออน สตูดิโอ) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  87220
for brand:  Crazy Toys(เครซี่ทอยส์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  1200
for brand:  Kitco Stainless(คิทโค่ สเตนเลส) Wi

for brand:  GRAND HOME(แกรนด์โฮม) Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  12285
for brand:  Inter National Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  570
for brand:  Biomax(ไบโอแม็กซ์) Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  6900
for brand:  Kanivore(คานิวอล) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  15450
for brand:  CATCHA (tofu cat litter)(ทรายแมวเต้าหู้ แคทช่า) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  23422
for brand:  Feliway(เฟลิเวย์) Winning cat 2 is:  Cat
with a cat_2_gmv value of:  5334
for brand:  Prins(ปริ๊น) Winning cat 2 is:  Spare parts and car accessories
with a cat_2_gmv value of:  2475
for brand:  Nikoyo(นิโคโย่) Winning cat 2 is:  Spare parts and car accessories
with a cat_2_gmv value of:  6054
for brand:  VOLTRONIC(โวลโทรนิก) Winning cat 2 is:  Car care products
with a cat_2_gmv value of:  4040
for brand:  Rupes(รูเพส) Winning cat 2 is:  Car care products
w

for brand:  Funtails Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  2800
for brand:  Centrum(เซนทรัม) Winning cat 2 is:  Health food supplements
with a cat_2_gmv value of:  1005487
for brand:  Esai(เอไซ) Winning cat 2 is:  Equipment for health and safety
with a cat_2_gmv value of:  690
for brand:  ZD Toys(ซีดี ทอยส์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  3690
for brand:  TULIP(ทิวลิป) Winning cat 2 is:  Other
with a cat_2_gmv value of:  13380
for brand:  Bocas(โบคาส) Winning cat 2 is:  Equipment for health and safety
with a cat_2_gmv value of:  2767
for brand:  Grand Home Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  900
for brand:  Super Mobile(ซูเปอร์ โมบาย) Winning cat 2 is:  Small kitchen appliances
with a cat_2_gmv value of:  9180
for brand:  Bandai Winning cat 2 is:  Game collectibles
with a cat_2_gmv value of:  600
for brand:  Mcdonald's(แมคโดนัลด์) Winning cat 2 is:  Game collectibles
with a cat_2_gmv value 

# Output: Brands that Meet Our Criterion

In [26]:
brands_df.sort_index(inplace=True)
brands_df

,Brand_GMV,Average_Unit_Price,GMV_Concentration,Weighted_Star_Rating,Bad_Rating_Percent,Brand_GMV_All_Cats,Conversation_Rate_All_Cats,Brand_Cat_2_Contr,Brand_Category_2
Brand Name,,,,,,,,,
3M,285462,313.350165,0.284048,4.778005,0.012,563616.0,6.032430,0.316,Car care products
ACCU-CHEK(แอคคิวเช็ค),684677,680.593439,0.665321,4.919250,0.002,1017173.0,38.517608,1.000,Health equipment
ACDelco(เอซีเดลโก้),18178,826.272727,0.775993,4.864057,0.002,333032.0,13.708970,0.909,Oil and liquid
AIKO(ไอโกะ),41810,459.450549,0.522865,4.782051,0.011,211626.0,6.813238,0.540,Gas stove
AJ(เอเจ),65799,715.206522,0.948571,4.570254,0.038,1573309.0,8.474780,0.302,Gas stove
...,...,...,...,...,...,...,...,...,...
sonar(โซนาร์),22349,698.406250,0.644548,4.663946,0.015,472764.0,12.167392,0.891,Refrigerator
xiaomi,52892,362.273973,1.000000,4.763268,0.017,697365.0,6.271381,0.252,Computer accessories
ดอกกรรณิการ์,359950,394.249726,0.886290,4.857046,0.006,359950.0,58.585612,1.000,Health equipment


In [27]:
brands_df.sort_values(by=["GMV_Concentration"], ascending=True).head(70)

,Brand_GMV,Average_Unit_Price,GMV_Concentration,Weighted_Star_Rating,Bad_Rating_Percent,Brand_GMV_All_Cats,Conversation_Rate_All_Cats,Brand_Cat_2_Contr,Brand_Category_2
Brand Name,,,,,,,,,
Royal canin(โรยัล คานิน),6228587,447.584579,0.057784,4.898229,0.004,11592942.0,34.181941,0.542,Dog
Bandai Spirits(บันไดสปิริท),359242,590.858553,0.070304,4.890834,0.002,769524.0,5.309699,1.000,Game collectibles
TOYOTA(โตโยต้า),554270,464.991611,0.103361,4.739633,0.012,2891769.0,8.983551,0.479,Car accessories
Meguiars(เมกไกวส์),439613,557.884518,0.129614,4.898733,0.004,540647.0,12.351717,0.997,Car care products
Triumph(ไทรอัมพ์),133413,670.417085,0.129837,4.518699,0.010,176299.0,11.122264,0.958,Underwear
...,...,...,...,...,...,...,...,...,...
Cherilon Intimate(เชอรีล่อน อินทิเมท),412758,403.083984,0.418856,4.839589,0.005,616822.0,6.324952,0.973,Underwear
Otto(ออตโต้),1752221,462.571542,0.418885,4.651314,0.023,1921680.0,9.221688,1.000,Small kitchen appliances
Good Smile Company(กู้ดสไมล์คัมปะนี),30791,1811.235294,0.418954,4.705882,0.000,457749.0,36.346594,1.000,Game collectibles


In [28]:
#make to csv here if desired 
#brands_df.to_csv("test_3.csv")

In [29]:
'''
mask = products_static_untouched["brand"] == "SKG(เอสเคจี)"
sample_brand_df = products_static_untouched[mask]
products_static_untouched[mask].sort_values("category_two_en", 
                                            ascending = False).tail(20)
'''


'\nmask = products_static_untouched["brand"] == "SKG(เอสเคจี)"\nsample_brand_df = products_static_untouched[mask]\nproducts_static_untouched[mask].sort_values("category_two_en", \n                                            ascending = False).tail(20)\n'

In [30]:
'''
winning_cat_2 = None
winning_cat_2_gmv = 0.0
    
for cat_2 in sample_brand_df["category_two_en"].unique(): 
    mask_cat_2 = sample_brand_df["category_two_en"] == cat_2
    cat_2_df = sample_brand_df[mask_cat_2]
    
    cat_2_gmv = cat_2_df["product_gmv"].sum()
    
    if cat_2_gmv > winning_cat_2_gmv:
        winning_cat_2 = cat_2
        winning_cat_2_gmv = cat_2_gmv
    
print(winning_cat_2, "with a cat gmv of: winning_cat_2_gmv")

'''

'\nwinning_cat_2 = None\nwinning_cat_2_gmv = 0.0\n    \nfor cat_2 in sample_brand_df["category_two_en"].unique(): \n    mask_cat_2 = sample_brand_df["category_two_en"] == cat_2\n    cat_2_df = sample_brand_df[mask_cat_2]\n    \n    cat_2_gmv = cat_2_df["product_gmv"].sum()\n    \n    if cat_2_gmv > winning_cat_2_gmv:\n        winning_cat_2 = cat_2\n        winning_cat_2_gmv = cat_2_gmv\n    \nprint(winning_cat_2, "with a cat gmv of: winning_cat_2_gmv")\n\n'

In [31]:
'''
To get category name info
models_static = pd.read_csv("models_static_data.csv")

mapping_1 = models_static[["category_one", "category_one_en", "category_one_th"]].drop_duplicates()
mapping_1 = mapping_1.set_index("category_one").dropna().sort_index()

mapping_2 = models_static[["category_two", "category_two_en", "category_two_th"]].drop_duplicates()
mapping_2 = mapping_2.set_index("category_two").dropna().sort_index()

mapping_3 = models_static[["category_three", "category_three_en", "category_three_th"]].drop_duplicates()
mapping_3 = mapping_3.set_index("category_three").dropna().sort_index()

mapping_1.to_csv("mapping_1.csv")
mapping_2.to_csv("mapping_2.csv")
mapping_3.to_csv("mapping_3.csv")
'''

'\nTo get category name info\nmodels_static = pd.read_csv("models_static_data.csv")\n\nmapping_1 = models_static[["category_one", "category_one_en", "category_one_th"]].drop_duplicates()\nmapping_1 = mapping_1.set_index("category_one").dropna().sort_index()\n\nmapping_2 = models_static[["category_two", "category_two_en", "category_two_th"]].drop_duplicates()\nmapping_2 = mapping_2.set_index("category_two").dropna().sort_index()\n\nmapping_3 = models_static[["category_three", "category_three_en", "category_three_th"]].drop_duplicates()\nmapping_3 = mapping_3.set_index("category_three").dropna().sort_index()\n\nmapping_1.to_csv("mapping_1.csv")\nmapping_2.to_csv("mapping_2.csv")\nmapping_3.to_csv("mapping_3.csv")\n'